In [43]:
import os
import glob
import librosa
from scipy.io import loadmat
from collections import defaultdict
import torch
from tqdm import tqdm
from torch import nn
import re

In [9]:
def get_song_info():

    song_dict = defaultdict(dict)

    songs = glob.glob("Metadata/**/*.txt", recursive=True)
    
    for file in tqdm(songs):
        with open(file) as f:
                key1 = file[file.index("/") + 1: file.rindex("/")]
                key2 = file[file.rindex("/") + 1:file.index(".")]
                lines = [line.rstrip("\n") for line in f]
                song_dict[key1][key2] = {"Metadata": lines}
                              
    
    return song_dict

Fs = 22050, hop size = 512

In [10]:
def get_audio_info(song_dict):

    mfcc_files = glob.glob("MFCCs/**/*.mat", recursive=True)

    num = len(mfcc_files)

    for mfcc_file in tqdm(mfcc_files, total=num):
        m = loadmat(mfcc_file)
        mfcc_matrix = m['XMFCC'].squeeze()
        mfcc_matrix = torch.tensor(mfcc_matrix, dtype=torch.float32)
        mean = mfcc_matrix.mean(dim=1)
        std = mfcc_matrix.std(dim=1)
        min_ = mfcc_matrix.min(dim=1).values
        max_ = mfcc_matrix.max(dim=1).values
        X = torch.cat([mean, std, min_, max_], dim = 0)

        key1 = mfcc_file[mfcc_file.index("/") + 1: mfcc_file.rindex("/")]
        key2 = mfcc_file[mfcc_file.rindex("/") + 1: mfcc_file.index("_")]
        song_dict[key1][key2]["X"] = X

    return song_dict



In [11]:
song_dict = get_song_info()

100%|██████████| 1000/1000 [00:00<00:00, 4968.54it/s]


In [12]:
print(song_dict["1"])

{'174702': {'Metadata': ["Come On Let's Go", 'Broadcast', 'March 20, 2000', 'First release']}, '464894': {'Metadata': ["Come On, Let's Go", 'Bart Davenport', 'January 21, 2011']}}


In [13]:
get_audio_info(song_dict)

100%|██████████| 1000/1000 [14:15<00:00,  1.17it/s]


defaultdict(dict,
            {'135': {'309237': {'Metadata': ['Where Country Is',
                'Dean Perritt',
                '2013'],
               'X': tensor([ 21.2385,  34.1055,  -0.7451,   7.0526,  -5.2623,  -0.2691,  -5.6482,
                        -4.3772,   0.5932,   1.6534,  -5.4091,  -1.5317,  -2.1636,   1.2429,
                        -4.3074,  -9.8807,   8.5004,  -4.2269,  -9.7762,  -7.8851,  13.5410,
                         5.9615,   7.9100,   8.5794,   7.0414,   6.8008,   7.6327,   8.6360,
                         7.7881,   7.4848,   6.6032,   7.0930,   7.1298,   7.8837,   6.8811,
                         6.8224,   7.9958,   9.1396,   7.5648,   7.5972, -47.2884,  11.3918,
                       -30.2483, -19.3152, -34.6741, -23.7173, -37.5115, -37.8306, -26.8957,
                       -29.5020, -31.3203, -24.7246, -35.1779, -28.7618, -39.9796, -29.8403,
                       -27.6736, -32.7223, -35.3798, -31.2553,  44.5306,  55.0284,  30.7417,
                  

In [14]:
print(song_dict['1']['174702'])
print(song_dict['1']['174702']['X'].shape)

{'Metadata': ["Come On Let's Go", 'Broadcast', 'March 20, 2000', 'First release'], 'X': tensor([-6.3495e+00,  2.2981e+01,  2.0622e+00,  1.6215e+01,  6.8331e+00,
         5.6570e+00,  1.1745e+01,  4.6823e+00,  2.9624e+00,  4.6705e+00,
        -3.9683e-01,  5.4030e+00,  3.7458e+00,  5.3102e+00,  3.4928e+00,
        -5.4979e-01, -2.0571e+00,  5.5056e+00, -3.2501e+00,  1.6432e+00,
         2.3008e+01,  5.7859e+00,  5.7019e+00,  6.7712e+00,  6.0518e+00,
         6.1356e+00,  6.5468e+00,  6.2972e+00,  6.6083e+00,  8.3437e+00,
         8.6695e+00,  7.5782e+00,  7.3402e+00,  7.0094e+00,  6.9483e+00,
         7.4929e+00,  9.5009e+00,  1.1088e+01,  1.1264e+01,  1.1476e+01,
        -2.3331e+02, -1.2900e-01, -2.2956e+01, -1.6809e+01, -1.6932e+01,
        -1.7187e+01, -9.2039e+00, -1.4061e+01, -2.5669e+01, -2.6141e+01,
        -2.9660e+01, -1.8124e+01, -1.9764e+01, -1.6352e+01, -1.5942e+01,
        -3.1355e+01, -2.7070e+01, -2.3673e+01, -4.2146e+01, -3.2998e+01,
         2.0729e+01,  4.0029e+01,  2

In [29]:
def create_feature_tensor(song_dict):

    X = torch.empty(0,80)

    for folder in sorted(song_dict):
        for song in sorted(song_dict[folder]):
            reshaped = song_dict[folder][song]["X"].view(1, -1)
            X = torch.cat([X, reshaped], dim=0)
    
    return X

In [30]:
X = create_feature_tensor(song_dict)

In [71]:
def group_songs_by_feature(song_dict):

    # songs grouped based on when the original was released
    first_release_groupings = {}

    # songs grouped based on when the cover was released
    cover_release_groupings = {}

    # songs grouped by number of covers released
    songs_with_more_than_3_covers = {}

    # first release songs mapped to their corresponding covers
    first_releases_to_covers = {}

    for folder in song_dict:
        if len(song_dict[folder]) >=4:
            songs_with_more_than_3_covers[folder] = [song_dict[folder][song]["Metadata"] for song in song_dict[folder]]
        for song in song_dict[folder]:
            if "First release" in song_dict[folder][song]["Metadata"]:
                first_releases_to_covers[song] = [tune for tune in song_dict[folder] if tune != song]
                year = song_dict[folder][song]["Metadata"][2]
                if "," in year:
                    year = year[year.index(","):]
                year = re.sub(r'[^0-9]', '', year)
                if f"{year[:3]}0" in first_release_groupings:
                    first_release_groupings[f"{year[:3]}0"].append(song)
                else:
                    first_release_groupings[f"{year[:3]}0"] = []
            else:
                year = song_dict[folder][song]["Metadata"][2]
                if "," in year:
                    year = year[year.index(","):]
                year = re.sub(r'[^0-9]', '', year)
                if f"{year[:3]}0" in cover_release_groupings:
                    cover_release_groupings[f"{year[:3]}0"].append(song)
                else:
                    cover_release_groupings[f"{year[:3]}0"] = []
    

    for key, val in first_release_groupings.items():
        first_release_groupings[key] = sorted(val)

    for key, val in cover_release_groupings.items():
        cover_release_groupings[key] = sorted(val)

    for key, val in songs_with_more_than_3_covers.items():
        songs_with_more_than_3_covers[key] = sorted(val)

    return (
        first_release_groupings,
        cover_release_groupings,
        songs_with_more_than_3_covers,
        first_releases_to_covers
    )


In [73]:
result = group_songs_by_feature(song_dict)
print(result[0].keys())
print(result[1].keys())
print(result[3]['174702'])

dict_keys(['1980', '2000', '1970', '1990', '2010', '1930', '1960', '1940', '1950'])
dict_keys(['2010', '1970', '1950', '1960', '1990', '2000', '1930', '1980', '1940', '1920', '1900'])
['464894']


['Pretty Young Thing', 'Stella Soleil', 'May 22, 2001', 'First release'] 338
